# PH 245 Final Project - Flu Absenteeism 

In [9]:
library(data.table)
install.packages("xlsx")
library(xlsx)

prefix = "../absentee/Combined-data/"
filenames = c("absentee_all.csv","absentee-flu.csv", "absentee-nonflu.csv", "ILIData_CA_201101_201739")

also installing the dependencies ‘rJava’, ‘xlsxjars’

Warning message in install.packages("xlsx"):
“installation of package ‘rJava’ had non-zero exit status”Warning message in install.packages("xlsx"):
“installation of package ‘xlsxjars’ had non-zero exit status”Warning message in install.packages("xlsx"):
“installation of package ‘xlsx’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


ERROR: Error in library(xlsx): there is no package called ‘xlsx’


In [2]:
# Loading Data (using high-speed data.tables for speed)
absenteeData = fread( file=paste(prefix, filenames[1], sep=""), stringsAsFactors=TRUE )

# Loading a smaller set of data for school-specific value aggregation
aggregationData = fread( file=paste(prefix, filenames[3], sep=""), stringsAsFactors=TRUE )

Read 42797568 rows and 9 (of 9) columns from 2.816 GB file in 00:00:51
Read 9274684 rows and 31 (of 31) columns from 1.633 GB file in 00:00:47


In [3]:
head(absenteeData)
colnames(absenteeData)

# Creating a smaller sample for use until final analysis
absenteeData = absenteeData[sample(.N, 10000)]
nrow(absenteeData)

schoolyr,date,grade,race,absent_nonill,absent_ill,dist,school,matchid
2011-12,29aug2011,0,African American,0,0,OUSD,ACORN Woodland Elementary,0
2011-12,29aug2011,0,African American,0,0,OUSD,ACORN Woodland Elementary,0
2011-12,29aug2011,0,African American,0,0,OUSD,ACORN Woodland Elementary,0
2011-12,29aug2011,0,Asian,0,0,OUSD,ACORN Woodland Elementary,0
2011-12,29aug2011,0,Latino,0,0,OUSD,ACORN Woodland Elementary,0
2011-12,29aug2011,0,Latino,0,0,OUSD,ACORN Woodland Elementary,0


[1] "schoolyr"      "date"          "grade"         "race"         
[5] "absent_nonill" "absent_ill"    "dist"          "school"       
[9] "matchid"

[1] 10000

In [4]:
head(aggregationData)
colnames(aggregationData)

V1,dist,school,schoolyr,date,grade,race,absent_nonill,absent_ill,matchid,⋯,per.englearn,per.freelunch,API13,API12,mean.cst.ela,per.adv.ela,per.basic.ela,mean.cst.m,per.adv.m,per.basic.m
1,OUSD,ACORN Woodland Elementary,NA,NA,NA,NA,NA,NA,NA,⋯,64.41948,86.70,798,848,360.600,22.50,26.5,392.025,35.50,11.75
2,OUSD,Allendale Elementary,2011-12,2011-08-29,0,African American,1,0,14,⋯,41.17647,79.91,663,725,329.625,8.75,38.5,333.425,10.75,29.00
3,OUSD,Allendale Elementary,2011-12,2011-08-29,0,African American,0,0,14,⋯,41.17647,79.91,663,725,329.625,8.75,38.5,333.425,10.75,29.00
4,OUSD,Allendale Elementary,2011-12,2011-08-29,0,African American,0,0,14,⋯,41.17647,79.91,663,725,329.625,8.75,38.5,333.425,10.75,29.00
5,OUSD,Allendale Elementary,2011-12,2011-08-29,0,African American,0,0,14,⋯,41.17647,79.91,663,725,329.625,8.75,38.5,333.425,10.75,29.00
6,OUSD,Allendale Elementary,2011-12,2011-08-29,0,African American,0,0,14,⋯,41.17647,79.91,663,725,329.625,8.75,38.5,333.425,10.75,29.00


[1] "V1"            "dist"          "school"        "schoolyr"     
 [5] "date"          "grade"         "race"          "absent_nonill"
 [9] "absent_ill"    "matchid"       "month"         "flusesn"      
[13] "dist.n"        "absent_all"    "enrolled"      "mn.class.size"
[17] "per.not_hsg"   "per.hsg"       "per.some_col"  "per.col_grad" 
[21] "per.grad_sch"  "per.englearn"  "per.freelunch" "API13"        
[25] "API12"         "mean.cst.ela"  "per.adv.ela"   "per.basic.ela"
[29] "mean.cst.m"    "per.adv.m"     "per.basic.m"

In [5]:
getSchoolData = function(aggregationData, dropColumns, aggregationColumns) {
    oldw <- getOption("warn")
    options(warn = -1)
    
    cleanAggregationData = aggregationData[,(dropColumns):=NULL]
    groupedSchoolData = cleanAggregationData[,head(.SD, 1),by=aggregationColumns]
    
    options(warn = oldw)

    return(groupedSchoolData)
}

In [6]:
# Dropping irrelevant columns from aggregation data
dropColumns = c("V1", "schoolyr", "date", "grade", "race", "absent_nonill", "absent_ill",
                "matchid", "month", "flusesn", "absent_all", "dist.n"
               )

aggregationColumns = c("dist", "school", "enrolled")

schoolData = getSchoolData(aggregationData, dropColumns, aggregationColumns)

print(paste("Data collected for", nrow(schoolData), "schools"))
head(schoolData)

[1] "Data collected for 84 schools"


dist,school,enrolled,mn.class.size,per.not_hsg,per.hsg,per.some_col,per.col_grad,per.grad_sch,per.englearn,per.freelunch,API13,API12,mean.cst.ela,per.adv.ela,per.basic.ela,mean.cst.m,per.adv.m,per.basic.m
OUSD,ACORN Woodland Elementary,267,19.07143,33,37,20,8,1,64.41948,86.70,798,848,360.600,22.50,26.50,392.025,35.50,11.75
OUSD,Allendale Elementary,425,26.56250,27,33,27,12,3,41.17647,79.91,663,725,329.625,8.75,38.50,333.425,10.75,29.00
OUSD,Bella Vista Elementary,525,21.87500,24,31,24,15,5,42.28571,75.33,813,849,369.825,29.75,22.75,422.575,53.50,12.00
OUSD,Bridges Academy,381,19.05000,55,30,11,3,1,79.26509,77.00,678,715,320.050,9.75,31.75,356.025,19.00,27.00
OUSD,Brookfield Village Elementary,367,16.68182,41,33,15,8,2,58.03815,66.21,687,738,329.675,8.25,44.00,351.575,18.25,22.50
OUSD,Burckhalter Elementary,298,22.92308,12,22,40,20,6,11.74497,71.81,769,808,358.950,22.50,30.75,390.775,39.00,21.00


In [7]:
# Cleaning data and adding more useful variables

absenteeData = absenteeData[,date:=as.Date(absenteeData$date, "%d%b%Y")]
absenteeData=absenteeData[,month:=as.numeric(format(absenteeData$date, "%m"))]
absenteeData=absenteeData[,week:=week(date)]
absenteeData=absenteeData[,yr:=year(date)]

absenteeData$fluseasCDC = ifelse(absenteeData$month <= 4 | absenteeData$month >= 10, 1, 0)

absenteeData$dist.n = ifelse(absenteeData$dist == "OUSD", 1, 0)

absenteeData$grade = as.factor(absenteeData$grade)

absenteeData$race <- factor(absenteeData$race, levels = c("White","African American",
      "Asian","Latino","Multiple Ethnicity","Native American","Not Reported",
      "Pacific Islander"))

# Since WCCUSD has different labeling and fewer races reported that OUSD, 
# reduce all races to subset for uniformity
absenteeData = absenteeData[race %in% c("Native American", "Multiple Ethnicity", "Not Reported"), 
                            race := "Don't know Other"]

# The sum of any row will be 0 if there was no absence 
# or 1 if there was an absence for any reason
absenteeData$absence = absenteeData$absent_nonill + absenteeData$absent_ill

# Marking all rows as in flu season or not using ILI definition:
# Start of flu season: at least 2 consecutive weeks in which % of medical visits for ILI is > 2%
# End of flu season: at least 2 consecutive weeks in which % of medical visits for ILI is < 2%
cdph=read.xlsx(file=paste(prefix, filenames[4], sep=""), sheetName="ILIData")

cdph$yr=substr(cdph$weekending,1,4)
cdph$mon=substr(cdph$weekending,6,7)
cdph$day=substr(cdph$weekending,9,10)


cdph$twoper=ifelse(cdph$ILIper>2,1,0)
cdph=cdph[order(cdph$weekending),]
cdph$fluseasCDPH=NA
cdph$fluseasCDPH[1]=1
for(i in 2:nrow(cdph)){
  cdph$fluseasCDPH[i][cdph$twoper[i]==1 & cdph$twoper[i-1]==1]=1
  cdph$fluseasCDPH[i][cdph$twoper[i]==0 & cdph$twoper[i-1]==0]=0
  cdph$fluseasCDPH[i][cdph$twoper[i]==0 & cdph$twoper[i-1]==1]=cdph$fluseasCDPH[i-1]
  cdph$fluseasCDPH[i][cdph$twoper[i]==1 & cdph$twoper[i-1]==0]=cdph$fluseasCDPH[i-1]
}

# Define peak week
cdph$seasno[(cdph$yr>=2011 & cdph$mon<6)]=1011
cdph$seasno[(cdph$yr>=2011 & cdph$mon>6)]=1112
cdph$seasno[(cdph$yr>=2012 & cdph$mon<6)]=1112
cdph$seasno[(cdph$yr>=2012 & cdph$mon>6)]=1213
cdph$seasno[(cdph$yr>=2013 & cdph$mon<6)]=1213
cdph$seasno[(cdph$yr>=2013 & cdph$mon>6)]=1314
cdph$seasno[(cdph$yr>=2014 & cdph$mon<6)]=1314
cdph$seasno[(cdph$yr>=2014 & cdph$mon>6)]=1415
cdph$seasno[(cdph$yr>=2015 & cdph$mon<6)]=1415
cdph$seasno[(cdph$yr>=2015 & cdph$mon>6)]=1516
cdph$seasno[(cdph$yr>=2016 & cdph$mon<6)]=1516
cdph$seasno[(cdph$yr>=2016 & cdph$mon>6)]=1617
cdph$seasno[(cdph$yr>=2017 & cdph$mon<6)]=1617

maxili=as.data.frame(aggregate(cdph$ILIper,list(seasno=cdph$seasno),max))
maxili.row=apply(as.matrix(maxili[,2]),1,function(x) which(cdph$ILIper==x))
cdph$peakwk=0
cdph$peakwk[maxili.row]=1
    
cdph$week=week(ymd(cdph$weekending))
    
cdph.sub=cdph[,c("weekending","fluseasCDPH","peakwk","week","yr")]
cdph.sub=as.data.table(cdph.sub)
cdph.sub$yr=as.integer(cdph.sub$yr)
    
# Merging flu season and peak week indicators into absentee dataset
cleanedAbsenteeData=cdph.sub[absenteeData,on=c("week","yr")]

# End result
head(absenteeData)
head(cleanedAbsenteeData)

ERROR: Error in eval(expr, envir, enclos): could not find function "read.xlsx"


#### Exploratory Data Analysis (EDA)
The first, most important thing to do is examine how many absences ocurred in total. Then, we'll break it down year by year and examine absences.

Absences are defined within the absent_nonill and absent_ill columns. Both columns having a 0 means the student was present. A 1 appears in one of the columns if there was an absence.

In examining our dataset, some other good things to understand include racial breakdown and grade distribution.

In [ ]:
# Beginning Exploratory Data Analysis
summary(absenteeData)

In [ ]:
pieAbsenceBreakdown = function(data, pieTitle) {
    "Creates a pie chart of the absences and presences in dataset"
    numAbsences = sum(data$absence)
    numPresences = length(data$absence) - numAbsences
    rawBreakdown = c(numAbsences, numPresences)
    
    piePercent = paste(round(100*rawBreakdown/sum(rawBreakdown), 2), "%", sep="")
    
    pie(rawBreakdown, 
        labels=piePercent, 
        col=rainbow(length(rawBreakdown)),
        main=pieTitle
       )
    
    legend("topright", 
           c("Absences","Presences"), 
           fill=rainbow(length(rawBreakdown))
          )
}

# Examining total absence/presence breakdown
pieAbsenceBreakdown(data=cleanAbsenteeData, pieTitle="All Year Absence/Presence breakdown")


# Examining flu-specific absence/presence breakdown
# fluData = cleanAbsenteeData[fluSeason==1]

#pieAbsenceBreakdown(data=fluData, pieTitle="Flu Season Absence/Presence breakdown")

In [ ]:
# Creating a pie chart of ethnicities

races = cleanAbsenteeData[,.N,by="race"]
piePercent2 = paste(round(100*races$N/sum(races$N), 2), "%", sep="")

pie(x=races$N, labels=piePercent2, col=rainbow(length(races$race)), cex = 0.4)
legend("topright", legend=races$race, fill=rainbow(length(races$race)), cex = 0.6, title="Ethnic breakdown")
races

In [ ]:
# Examining overall grade distribution
grades = cleanAbsenteeData[,.N,by="grade"][order(grade)]

barplot(grades$N, names.arg=grades$grade)

#### Interpreting Our EDA Results

So, we see that we have a relatively small number of absences in our overall dataset (this is good!). Since we have a huge sample size, we'll have plenty of absences to examine.

Let's perhaps use PCA to interpret whether race is a good predictor of absences? Or whether the school's stats are?

Predicting school level absence based on all of its variables in nonflusesn and in flusesn and based on whether it received the intervention or not? 